<a href="https://colab.research.google.com/github/eyaler/avatars4all/blob/master/yarok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# U^2-Net Video Green Screen
## https://github.com/NathanUA/U-2-Net
### Made just a little bit more accessible by Eyal Gruss (https://eyalgruss.com, eyalgruss@gmail.com)

#### Foreground options:
*   Video from web or uploaded
*   Mirrored versions of the above

#### Mask options:
*   Continuous blending
*   One frame delay smoothing with custom threshold (https://arxiv.org/pdf/2011.11961)

#### Background options:
*   Image from web or uploaded
*   Video from web or uploaded
*   Mirrored versions of the above
*   Solid black, white or green
*   Foreground video converted to grayscale

##### List of more generative tools: https://j.mp/generativetools



In [ ]:
#@title Setup
%cd /content
!git clone --depth 1 https://github.com/NathanUA/U-2-Net
!mkdir -p /content/U-2-Net/saved_models/u2net
%cd /content/U-2-Net/saved_models/u2net
!wget --no-check-certificate -nc https://eyalgruss.com/fomm/u2net.pth
%cd /content
!pip install -U youtube-dl
!pip install -U imageio
!pip install -U imageio-ffmpeg

In [ ]:
#@title Get the foreground video and background image/video from the web
#@markdown 1. You can change the URLs to your **own** stuff!
#@markdown 2. For the background image you can use the drop-down menu to alternatively choose a **solid color** or a **greyscale** version of the foreground video
#@markdown 3. Alternatively, you can upload **local** files in the next cells

video_url = 'https://www.youtube.com/watch?v=kMpnwIGDQvU' #@param {type:"string"}
background_url = 'https://cdnb.artstation.com/p/assets/images/images/012/476/185/large/rafael-de-jongh-synthwave-neon-80s-background-marmoset-revamp-compressed.jpg' #@param ['https://cdnb.artstation.com/p/assets/images/images/012/476/185/large/rafael-de-jongh-synthwave-neon-80s-background-marmoset-revamp-compressed.jpg', 'https://www.youtube.com/watch?v=2yy1-xXMI-w', 'Black', 'White', 'Green', 'Gray video'] {allow-input: true}

if video_url:
  !rm -f /content/video.mp4
  !youtube-dl -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4' '$video_url' --merge-output-format mp4 -o /content/video
  !mv /content/video.mp4 /content/video 
  duration_seconds = None
  params = ''

import youtube_dl
def is_supported(url):
    extractors = youtube_dl.extractor.gen_extractors()
    for e in extractors:
        if e.suitable(url) and e.IE_NAME != 'generic':
            return True
    return False

if '://' in background_url:
  if is_supported(background_url):
    !rm -f /content/background.mp4
    !youtube-dl -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4' '$background_url' --merge-output-format mp4 -o /content/background
    !mv /content/background.mp4 /content/background
    bg_params = ''
  else:
    !wget '$background_url' -O /content/background

In [ ]:
#@title Optionally upload foreground video { run: "auto" }
manually_upload_video = False #@param {type:"boolean"}
if manually_upload_video:
  from google.colab import files
  import shutil

  %cd /content/sample_data
  try:
    uploaded = files.upload()
  except Exception as e:
    %cd /content
    raise e

  for fn in uploaded:
    shutil.move('/content/sample_data/'+fn, '/content/video')
    break
  params = ''

In [ ]:
#@title Optionally upload local background image/video { run: "auto" }
manually_upload_background = False #@param {type:"boolean"}
if manually_upload_background:
  from google.colab import files
  import shutil

  %cd /content/sample_data
  try:
    uploaded = files.upload()
  except Exception as e:
    %cd /content
    raise e

  for fn in uploaded:
    shutil.move('/content/sample_data/'+fn, '/content/background')
    break
  background_url = None
  bg_params = ''

In [ ]:
#@title Optionally shorten foreground video
start_seconds =  0#@param {type:"number"}
duration_seconds =  60#@param {type:"number"}
start_seconds = max(start_seconds,0)
duration_seconds = max(duration_seconds,0)
params = '-ss %i -t %i'%(start_seconds, duration_seconds)

In [ ]:
#@title Optionally shorten background video
background_start_seconds =  0#@param {type:"number"}
background_duration_seconds =  60#@param {type:"number"}
background_start_seconds = max(background_start_seconds,0)
background_duration_seconds = max(background_duration_seconds,0)
bg_params = '-ss %i -t %i'%(background_start_seconds, background_duration_seconds)

In [ ]:
#@title Green Screen It!

#https://arxiv.org/pdf/2011.11961
one_frame_delay = True #@param {type:"boolean"} 
one_frame_delay_threshold = 0.1 #@param {type:"slider", min:0, max:1, step:0.05}
mirror_foreground = False #@param {type:"boolean"} 
mirror_background = False #@param {type:"boolean"} 

frames_dir = '/content/U-2-Net/test_data/test_images'
mask_dir = '/content/U-2-Net/test_data/u2net_results'
bg_dir = '/content/bg_frames'
out_dir = '/content/out_frames'
!rm -rf $frames_dir
!mkdir -p $frames_dir
!rm -rf $mask_dir
!mkdir -p $mask_dir
!rm -rf $bg_dir
!mkdir -p $bg_dir
!rm -rf $out_dir
!mkdir -p $out_dir
!ffmpeg $params -i /content/video $frames_dir/frame_%05d.png

from skimage.transform import resize
import imageio
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")

def fix_dims(im):
    if im.ndim == 2:
        im = np.tile(im[..., None], [1, 1, 3])
    return im[...,:3]

target_image = imageio.imread(frames_dir+'/frame_00001.png')

if background_url == 'Black':
  source_image = np.full_like(target_image, 0)
elif background_url == 'White':
  source_image = np.full_like(target_image, 1)
elif background_url == 'Green':
  source_image = np.full_like(target_image, [0,1,0])
elif background_url != 'Gray video':
  try:
    source_image = imageio.imread('/content/background')
    source_image = fix_dims(source_image)
    source_image = resize(source_image, target_image.shape[:2])
    if mirror_background:
      source_image = np.fliplr(source_image)
  except Exception:
    !ffmpeg $bg_params -i /content/background $bg_dir/frame_%05d.png

%cd /content/U-2-Net
!python /content/U-2-Net/u2net_test.py

files = [x for x in sorted(os.listdir(frames_dir)) if x.endswith('.png')]
bg_files = [x for x in sorted(os.listdir(bg_dir)) if x.endswith('.png')]
mask_plus = None
for i,file in enumerate(files):
    im = imageio.imread(frames_dir+'/'+file)/255
    if one_frame_delay and i>0 and i<len(files)-1:
      mask_minus = mask_now
      mask_now = mask_plus
      if mask_now is None:
        mask_now = imageio.imread(mask_dir+'/'+file)/255
      mask_plus = imageio.imread(mask_dir+'/'+files[i+1])/255
      cond = (np.abs(mask_plus-mask_minus)<=one_frame_delay_threshold) & (np.abs(mask_now-mask_minus)>one_frame_delay_threshold) & (np.abs(mask_now-mask_plus)>one_frame_delay_threshold)
      mask = mask_now*(1-cond) + (mask_minus+mask_plus)/2*cond
    else:
      mask_now = imageio.imread(mask_dir+'/'+file)/255
      mask = mask_now
    if mirror_foreground:
        im = np.fliplr(im)
        mask = np.fliplr(mask)
    if background_url == 'Gray video':
      source_image = fix_dims(np.dot(im, [0.2989, 0.5870, 0.1140]))
    elif bg_files:
      source_image = imageio.imread(bg_dir+'/'+bg_files[i%len(bg_files)])
      source_image = resize(source_image, target_image.shape[:2])
      if mirror_background:
        source_image = np.fliplr(source_image)
    imageio.imwrite(out_dir+'/'+file, np.uint8((source_image*(1-mask)+im*mask)*255))

with imageio.get_reader('/content/video', format='mp4') as reader:
  fps = reader.get_meta_data()['fps']

!ffmpeg -framerate $fps -i $out_dir/frame_%05d.png $params -i /content/video -c:v libx264 -pix_fmt yuv420p /content/final.mp4 -y
#video can be downloaded from /content/final.mp4

from IPython.display import HTML, clear_output
from base64 import b64encode

#clear_output()
with open('/content/final.mp4', 'rb') as f:
  data_url = "data:video/mp4;base64," + b64encode(f.read()).decode()
display(HTML("""
<video width=600 controls autoplay loop>
      <source src="%s" type="video/mp4">
</video>""" % data_url))

In [ ]:
#@title Download
#@markdown 1. If it fails try running this cell again.
#@markdown 2. Alternatively, you can manually download "final.mp4" from the folder on the left (click "Refresh" if missing).

print() #see https://github.com/googlecolab/colabtools/issues/468
from google.colab import files
files.download('/content/final.mp4') #fails for Firefox private window